<a href="https://colab.research.google.com/github/Lord0fTheFlies/NMA-DL-Project/blob/main/Copy_of_BioBERT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install necessory libraries**

In [ ]:
!pip install datasets
!pip install wandb
!pip install transformers[torch] --upgrade
!pip install accelerate --upgrade
!pip install peft
!pip install sacremoses
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

**CPU or GPU :/**

In [ ]:
import torch

if torch.cuda.is_available():
    print("Running on GPU")
    print(torch.cuda.get_device_name(0))
else:
    print("Running on CPU")


Running on GPU
Tesla T4


**Import the necessary modules**

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import os
from transformers import AutoTokenizer, BioGptForSequenceClassification, EvalPrediction, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
import torch.optim as optim
from transformers import DataCollatorWithPadding
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from datasets import Dataset
from sklearn.model_selection import train_test_split
import wandb
import numpy as np
from datasets import load_dataset
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from datasets import Dataset, Features, Array3D, Array2D
import evaluate
from datasets import load_metric
from transformers import Trainer, TrainingArguments


**Import drive**

MessageError: Error: credential propagation was unsuccessful

**Read symptom disease from driv**

In [ ]:
# uploaded Data set
df = pd.read_csv('LLM_df_stringify_filtered.csv')

In [ ]:
print(df.info())
df.sample(n=5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26350 entries, 0 to 26349
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      26350 non-null  int64 
 1   diseases        26350 non-null  object
 2   Symptom_labels  26350 non-null  object
dtypes: int64(1), object(2)
memory usage: 617.7+ KB
None


,Unnamed: 0,diseases,Symptom_labels
4331,4331,chickenpox,"cough, pain in testicles, abnormal appearing s..."
10665,10665,gynecomastia,"skin swelling, back pain, neck pain, problems ..."
26178,26178,vulvar disorder,"vaginal itching, pain during intercourse, vagi..."
26186,26186,vulvar disorder,"pain during intercourse, vaginal pain, vulvar ..."
13464,13464,injury to the trunk,"sharp chest pain, headache, wrist pain, back p..."


**Pre Process the data before train model**

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# Assuming the dataset has 'symptoms' and 'disease' columns
symptoms = df['Symptom_labels'].tolist()
diseases = df['diseases'].tolist()

# Encode the labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
disease_labels = le.fit_transform(diseases)

# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(symptoms, disease_labels, test_size=0.2)

# Load BioBERT tokenizer
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

**Tokenize the data set with loaded BioBERT tokenizer**

In [ ]:
def tokenize_function(texts):
    return tokenizer(texts, truncation=True, padding='max_length', max_length=200) ###


train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)


In [ ]:
train_encodings.items()

**Convert the tokenized data into PyTorch datasets.**


In [ ]:
import torch

class DiseaseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = DiseaseDataset(train_encodings, train_labels)
val_dataset = DiseaseDataset(val_encodings, val_labels)


In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.1', num_labels=len(le.classes_))


**Compute accuracy**

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
    }


****

**Set up the training arguments and trainer.**

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    #evaluation_strategy="epoch",
    num_train_epochs=4,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    fp16=True,
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    learning_rate=2e-5,  # Adjust this value, e.g., try 3e-5 or 1e-5
    #load_best_model_at_end=True,


)




trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics      # compute metrics function

)


**Train the model**

In [ ]:
trainer.train()

**Reporting results including accuracy**

In [ ]:
results = trainer.evaluate()
print(results)